In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Change directory
%cd /content/drive/My Drive/CISC_CMPE351

# Load file into pandas data frame
myDataFrame = pd.read_csv('10dayWindow_original_no_duplicates_date_tick.csv')

# Drop rows with NaN
myDataFrame.dropna(inplace=True)

# Convert stock price movement to % change
for i in range(2, 22):
    myDataFrame['adjclose_{}_Y'.format(i)] = (myDataFrame['adjclose_{}_Y'.format(i)] - myDataFrame['adjclose_1_X'])/myDataFrame['adjclose_1_X']

# Get 80% of the data by date
myDataFrame['date'] = pd.to_datetime(myDataFrame['date'])
myDataFrame = myDataFrame.sort_values('date', ascending = False)
myDataFrame = myDataFrame.reset_index(drop=True)
dates = pd.to_datetime(myDataFrame['date'])
myDataFrame = myDataFrame.drop(columns = ['date', 'ticker'])
(dates <= dates.quantile(.8)).idxmax()

# Most recent data for testing
test =  myDataFrame.head(49869)

# Rest of the data is training
train = myDataFrame.tail(-49869)

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1z4HnqdQQElB4DgNtOX0BZ1t5Ge-L6W5i/CISC_CMPE351


In [ ]:
#Run this if using both Reddit and Financial data
X_train = train.iloc[:, list(range(0,120))+list(range(240,246))] 
y_train = train.iloc[:, list(range(124, 244, 6))] 
X_test = test.iloc[:, list(range(0,120))+list(range(240,246))]
y_test = test.iloc[:, list(range(124, 244, 6))]

In [ ]:
#Run this if only using Reddit data
X_train = train.iloc[:, list(range(240,246))] 
y_train = train.iloc[:, list(range(124, 244, 6))] 
X_test = test.iloc[:, list(range(240,246))]
y_test = test.iloc[:, list(range(124, 244, 6))]

In [ ]:
indim = X_train.shape[1]
outdim = y_train.shape[1]

#Definte Feed-Forward Neural Network
model = Sequential()

model.add(Dense(40, input_dim=indim, activation="softmax"))
model.add(Dense(20, activation="softmax"))
model.add(Dense(outdim))

# Compile the model
model.compile(optimizer='rmsprop', loss='mse')

#Fit the model to training data
model.fit(X_train,y_train, epochs=10, batch_size=20)

#Make predictions
ypred = model.predict(X_test)

Epoch 1/10
10021/10021 [==============================] - 13s 1ms/step - loss: 11861973.8065
Epoch 2/10
10021/10021 [==============================] - 13s 1ms/step - loss: 17211284.7688
Epoch 3/10
10021/10021 [==============================] - 13s 1ms/step - loss: 5062415.8006
Epoch 4/10
10021/10021 [==============================] - 13s 1ms/step - loss: 23383781.2485
Epoch 5/10
10021/10021 [==============================] - 12s 1ms/step - loss: 21161869.4345
Epoch 6/10
10021/10021 [==============================] - 12s 1ms/step - loss: 20463729.9371
Epoch 7/10
10021/10021 [==============================] - 13s 1ms/step - loss: 6103673.4852
Epoch 8/10
10021/10021 [==============================] - 19s 2ms/step - loss: 25458596.5973
Epoch 9/10
10021/10021 [==============================] - 13s 1ms/step - loss: 18503865.0453
Epoch 10/10
10021/10021 [==============================] - 13s 1ms/step - loss: 38235636.4352


In [ ]:
#Save the predictions and true results to csv's
np.savetxt(r"NN_10W_Reddit_ytest.csv", y_test, delimiter=",")
np.savetxt(r"NN_10W_Reddit_ypred.csv", ypred, delimiter=",")